# Quizz4

In [1]:
from pyspark import SparkContext
sc = SparkContext()
sc.addPyFile("graphframes-0.7.0-spark2.4-s_2.11.jar")

from graphframes import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()
spark

## Question 1

In [2]:
# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
|  g|  e|      follow|
+---+---+------------+



In [3]:
Alice_two_hop = g.find("(a)-[]->(b); (b)-[]->(c)") \
                 .filter("a.name = 'Alice'")
Alice_two_hop.show()

+--------------+---------------+----------------+
|             a|              b|               c|
+--------------+---------------+----------------+
|[a, Alice, 34]|[e, Esther, 32]|  [f, Fanny, 38]|
|[a, Alice, 34]|[e, Esther, 32]|  [d, David, 29]|
|[a, Alice, 34]|   [b, Bob, 36]|[c, Charlie, 37]|
+--------------+---------------+----------------+



In [4]:
Alice_two_hop.select("c.name") \
             .show()

+-------+
|   name|
+-------+
|  Fanny|
|  David|
|Charlie|
+-------+



In [5]:
# Answer Sheet
Alice_two_hop = g.find("(a)-[]->(b); (b)-[]->(c)").filter("a.name = 'Alice'")
Alice_two_hop.select("c.name").show()

+-------+
|   name|
+-------+
|  Fanny|
|  David|
|Charlie|
+-------+



## Question 2

In [6]:
Alice_two_hop_back = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)") \
                      .filter("a.name = 'Alice'")
Alice_two_hop_back.show()

+--------------+---------------+--------------+
|             a|              b|             c|
+--------------+---------------+--------------+
|[a, Alice, 34]|[e, Esther, 32]|[d, David, 29]|
+--------------+---------------+--------------+



In [7]:
Alice_two_hop_back.select("c.name") \
                  .show()

+-----+
| name|
+-----+
|David|
+-----+



In [8]:
# Answer Sheet

Alice_two_hop_back = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(a)").filter("a.name = 'Alice'")
Alice_two_hop_back.select('c.name').show()

+-----+
| name|
+-----+
|David|
+-----+



## Question 3

In [9]:
who_follow_Charie = g.find("(a)-[e]->(b)") \
                     .filter("b.name = 'Charlie' AND e.relationship = 'follow'")
who_follow_Charie.show()          

+--------------+--------------+----------------+
|             a|             e|               b|
+--------------+--------------+----------------+
|[f, Fanny, 38]|[f, c, follow]|[c, Charlie, 37]|
|  [b, Bob, 36]|[b, c, follow]|[c, Charlie, 37]|
+--------------+--------------+----------------+



In [10]:
who_follow_Charie.select("a.name") \
                 .show()

+-----+
| name|
+-----+
|Fanny|
|  Bob|
+-----+



In [11]:
# Answer Sheet
who_follow_Charie = g.find("(a)-[e]->(b)") \
                     .filter("b.name = 'Charlie' AND e.relationship = 'follow'")
who_follow_Charie.select("a.name").show()

+-----+
| name|
+-----+
|Fanny|
|  Bob|
+-----+



## Question 4

In [4]:
e2 = g.edges.filter("relationship = 'follow'")
g2 = GraphFrame(v, e2)
g2.vertices.show()
g2.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 37|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 38|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  g|  e|      follow|
+---+---+------------+



In [15]:
g2.vertices.join(g.inDegrees, 'id', 'left_outer') \
          .where("inDegree >= 2") \
          .select("name") \
          .show()

+-------+
|   name|
+-------+
| Esther|
|Charlie|
|    Bob|
+-------+



In [14]:
# Answer Sheet
e2 = g.edges.filter("relationship = 'follow'")
g2 = GraphFrame(v, e2)
g2.vertices.join(g2.inDegrees, 'id', 'left_outer') \
           .where("inDegree >= 2") \
           .select("name") \
           .show()

+-------+
|   name|
+-------+
| Esther|
|Charlie|
|    Bob|
+-------+



In [5]:
# Answer Sheet
e2 = g.edges.filter("relationship = 'follow'")
g2 = GraphFrame(v, e2)
g2.vertices.join(g2.inDegrees, 'id', 'left_outer') \
           .where("inDegree >= 2") \
           .select("name") \
           .show()

+-------+
|   name|
+-------+
|Charlie|
+-------+



## Question 5

In [2]:
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('data/adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newNoun, runningNoun):
    if runningNoun is None:
        return newNoun
    elif len(newNoun) > len(runningNoun):
        return newNoun
    else:
        return runningNoun

pairs = lines.map(lambda l: tuple(l.split(" "))) \
             .filter(lambda p: len(p)==2) \
             .reduceByKey(lambda a, b: a if len(a) > len(b) else b) \
             .updateStateByKey(updateFunc)

def printResults(rdd):
    print (rdd.take(5))
    print ('Longest noun associated with good:', rdd.lookup('good')[0])

pairs.foreachRDD(printResults)

In [3]:
ssc.start()
print("Start")
ssc.awaitTermination(150)
ssc.stop(False)
print("Finished")

Start
[('social', ['constructionism']), ('coercive', ['deprogramming']), ('individual', ['meaninglessness']), ('political', ['self-determination']), ('desirable', ['characteristic'])]
Longest noun associated with good: ['morning/afternoon']
[('social', ['constructionism']), ('coercive', ['deprogramming']), ('individual', ['meaninglessness']), ('political', ['self-determination']), ('desirable', ['characteristic'])]
Longest noun associated with good: ['morning/afternoon']
[('social', ['constructionism']), ('coercive', ['deprogramming']), ('individual', ['meaninglessness']), ('political', ['self-determination']), ('desirable', ['characteristic'])]
Longest noun associated with good: ['morning/afternoon']
[('social', ['constructionism']), ('coercive', ['deprogramming']), ('individual', ['meaninglessness']), ('political', ['self-determination']), ('desirable', ['characteristic'])]
Longest noun associated with good: ['morning/afternoon']
[('social', ['constructionism']), ('coercive', ['depro

In [ ]:
# Answer Sheet
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('data/adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newNoun, runningNoun):
    if runningNoun is None:
        return newNoun
    elif len(newNoun) > len(runningNoun):
        return newNoun
    else:
        return runningNoun

pairs = lines.map(lambda l: tuple(l.split(" "))) \
             .filter(lambda p: len(p)==2) \
             .reduceByKey(lambda a, b: a if len(a) > len(b) else b) \
             .updateStateByKey(updateFunc)

def printResults(rdd):
    print (rdd.take(5))
    print ('Longest noun associated with good:', rdd.lookup('good')[0])

pairs.foreachRDD(printResults)

ssc.start()
print("Start")
ssc.awaitTermination(150)
ssc.stop(False)
print("Finished")